In [3]:
# from flask_other.app import app
from flask import Flask
import os,sys
sys.path.append('../')
app=Flask("demo")
from flask import  render_template,request
from app.mong_database import MongoManager
import time
import re
mongo_db=MongoManager(server_ip='chatbotdb')

import urllib.request
from urllib.parse import quote
import pandas as pd
import numpy as np
import time
import re
import requests
import uuid

##################################
url = 'http://localhost:8889/chatbotv1'


def get_data(sentence, uid):
    sessionId = cache.uid_dict[uid]
    response = requests.get(url, timeout=10, params={'data':sentence, 
                                                     'action':'chat', 
                                                     'sessionId':sessionId})
    
    if response.status_code == 200:
        req = response.json()
        if req.get('status').lower() == 'successful':
            msg = req.get('message')
        else:
            msg = req

        return msg
    else:
        return 'Internet Connection Issue, error code {}'.format(response.status_code)



@app.route('/')
@app.route('/index')
def index0():
    return render_template(
                           # "index0.html",
                           "index.html",
                           )

@app.route("/createCookie", methods = ['POST'])
def createCookie():
    if request.method == 'POST':
        
        cookieid = request.form.get('cookie');
        if cookieid:
            print('already exists')
            print(cookieid)
            return('')
        else:
            print('new user')
            uid = str(uuid.uuid1())
            print('create cookie {}'.format(uid))
            return(str(uuid.uuid1()))



@app.route("/predict", methods= ["POST"])
def background_process():
    if request.method == 'POST':
        try:
            query = request.form.get('query')#前端查询的内容
            uid = request.form.get('uid')
            print(uid)
            if query:

                    
                    # time.sleep(5)
                    result = get_data(query, uid)
                    mongo_db.save_query(query, str(result))
                    print('saving to mongo db successfully!')
                    return str(result)
                    

            else:

                    return str('请输入查询内容')


        except Exception as e:
            print(e)

            if 'duplicate' in str(e):
                e_str = e.details['errmsg']
                dup_id=re.search('\{ : "(.*)" \}',e_str).group(1)
                print('query duplicated! Use same id to save')
                mongo_db.update_dup_query( dup_id, str(result))
                return str(result)

            else:
                print(e)
                return str('MM出故障啦')


    else:
        return 'ok'
    
    
@app.route("/newConversation", methods= ["POST"])
def new_conversation():
    
    print('new conversation called!!!!!!!!!!!!!!!!!!!!!!!')
    if request.method == 'POST':
        cookie = request.form.get('uid');
        if cookie:
            print(cookie)
            cache.new_conversation(cookie)
            return cookie
        else:
            print('cookie lost')
            return('error')

class Cache:
    def __init__(self):
#         self.new_conversation()
        self.uid_dict = {}
            
    def new_conversation(self):
        req = requests.get(url, timeout=10, params={'action':'create'})
        if req.status_code == 200:
            msg = req.json()
            if msg['status'] == 'successful':
                
                self.sessionId = msg['message']['sessionId']
                print(self.sessionId)
            else:
                
                raise ValueError('cannot get new sessionId, cannot start chat')
        else:
            print(req.status_code)
            
    def new_conversation(self, uid):
        req = requests.get(url, timeout=10, params={'action':'create'})
        if req.status_code == 200:
            msg = req.json()
            if msg['status'] == 'successful':
                sessionId = msg['message']['sessionId']
                self.uid_dict[uid] = sessionId
                print(self.uid_dict[uid])
            else:
            
                raise ValueError('cannot get new sessionId, cannot start chat')
        else:
            print(req.status_code)
            
        


        
cache = Cache()   




chatbotdb


In [4]:
app.debug = False
port = 6006
print('running at http://10.0.24.31:{}'.format(port))
app.run(host='0.0.0.0',port=port)    #这样用来监听所有的ip，团队调试用

 * Running on http://0.0.0.0:6006/ (Press CTRL+C to quit)


running at http://10.0.24.31:6006


172.18.0.1 - - [19/Jun/2018 05:39:59] "GET / HTTP/1.1" 200 -
172.18.0.1 - - [19/Jun/2018 05:40:02] "POST /newConversation HTTP/1.1" 200 -
172.18.0.1 - - [19/Jun/2018 05:40:02] "POST /createCookie HTTP/1.1" 200 -
172.18.0.1 - - [19/Jun/2018 05:40:02] "GET / HTTP/1.1" 200 -


new conversation called!!!!!!!!!!!!!!!!!!!!!!!
bd50c2ca-733e-11e8-ace5-0242ac120002
1
already exists
bd50c2ca-733e-11e8-ace5-0242ac120002


172.18.0.1 - - [19/Jun/2018 05:40:05] "POST /dataFromAjax_post HTTP/1.1" 404 -
172.18.0.1 - - [19/Jun/2018 05:40:05] "POST /predict HTTP/1.1" 200 -


...

new conversation called!!!!!!!!!!!!!!!!!!!!!!!
4cdf92d8-7383-11e8-b435-0242ac120002
2


172.18.0.1 - - [19/Jun/2018 05:41:08] "GET /static/avatar1.jpeg HTTP/1.1" 200 -
172.18.0.1 - - [19/Jun/2018 05:41:08] "POST /dataFromAjax_post HTTP/1.1" 404 -
172.18.0.1 - - [19/Jun/2018 05:41:08] "POST /predict HTTP/1.1" 200 -
172.18.0.1 - - [19/Jun/2018 05:41:08] "GET /static/avatar2.jpeg HTTP/1.1" 200 -


4cdf92d8-7383-11e8-b435-0242ac120002
query 你好
result 你好，这里是H催收公司，请问是罗巍先生吗？
E11000 duplicate key error collection: flask_database.query index: _id_ dup key: { : "7eca689f0d3389d9dea66ae112e5cfd7" }
query duplicated! Use same id to save


172.18.0.1 - - [19/Jun/2018 05:41:11] "POST /dataFromAjax_post HTTP/1.1" 404 -
172.18.0.1 - - [19/Jun/2018 05:41:11] "POST /predict HTTP/1.1" 200 -


4cdf92d8-7383-11e8-b435-0242ac120002
query 是啊
result 你好，我是H催收公司的客服小催，您之前借贷了H公司5万块钱，约定在2018年5月1日还清。您已还款3万2千块钱，但因逾期未还产生了相应的利息和延迟还款费用，现在一共需要还2万块钱，其中1.2万本金，1500利息，500延迟还款手续费用，请问您打算什么时候处理下呢？
E11000 duplicate key error collection: flask_database.query index: _id_ dup key: { : "74b4f86de2cec4138e234f9f1af02b2e" }
query duplicated! Use same id to save
